In [ ]:
import cv2
import os

def extract_and_save_scenes(video_path, output_dir):
    """
    Extracts one frame per second from the provided video, saves them as image files locally,
    and returns scenes as a dictionary.

    Parameters:
    video_path (string): Path of the video to be processed.
    output_dir (string): Directory where the extracted frames will be saved.

    Returns:
    list of dict: Each dictionary corresponds to a scene, containing:
        - index (int): Scene index
        - img_file (string): File name of the saved image
    """
    
    if not os.path.exists(video_path):
        print(f"{video_path}: File not found.")
        return []

    # Create output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Open the video file
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    if fps <= 0:
        print(f"Error: Could not retrieve FPS from {video_path}")
        return []

    scenes = []
    index = 0
    success, frame = cap.read()
    count = 0

    while success:
        # Capture frame every second (approximately)
        if count % int(fps) == 0:
            # Define the file name for the saved image
            img_file = os.path.join(output_dir, f"scene_{index}.jpg")
            
            # Save the frame as an image file
            cv2.imwrite(img_file, frame)
            
            # Append scene info to the list
            scenes.append({
                "index": index,
                "img_file": img_file
            })
            
            index += 1
        
        success, frame = cap.read()
        count += 1

    cap.release()
    
    return scenes

In [ ]:
import os
import re
import cv2
import numpy as np
import pandas as pd
from numpy.linalg import norm
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Load VGG16 model (no top layer, just convolutional base)
vgg_model = VGG16(weights='imagenet', include_top=False)

# Extract numeric part from filenames like scene_23.jpg
def extract_frame_number(filename):
    match = re.search(r'(\d+)', filename)
    return int(match.group(1)) if match else -1

# CNN image preprocessing
def preprocess_image(image_path):
    img = cv2.imread(image_path)
    if img is None:
        raise Exception(f"Cannot read image: {image_path}")
    img = cv2.resize(img, (224, 224))
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    return img

# CNN distance between two images
def compute_cnn_distance(img_path1, img_path2):
    img1 = preprocess_image(img_path1)
    img2 = preprocess_image(img_path2)
    feat1 = vgg_model.predict(img1).flatten()
    feat2 = vgg_model.predict(img2).flatten()
    return norm(feat1 - feat2)

# Modified grouping function using CNN
def group_frames_by_cnn_diff(input_folder, output_folder, threshold):
    os.makedirs(output_folder, exist_ok=True)

    # Sorted image files by number
    image_files = sorted([
        os.path.join(input_folder, f)
        for f in os.listdir(input_folder)
        if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp'))
    ], key=lambda x: extract_frame_number(os.path.basename(x)))

    if len(image_files) < 2:
        print("Not enough images to compute difference.")
        return

    subfolder_idx = 1
    subfolder_path = os.path.join(output_folder, f"sub{subfolder_idx}")
    os.makedirs(subfolder_path, exist_ok=True)

    diff_log = []
    prev_img_path = image_files[0]
    cv2.imwrite(os.path.join(subfolder_path, os.path.basename(prev_img_path)), cv2.imread(prev_img_path))

    for i in range(1, len(image_files)):
        curr_img_path = image_files[i]
        distance = compute_cnn_distance(prev_img_path, curr_img_path)

        diff_log.append({
            'frame1': os.path.basename(prev_img_path),
            'frame2': os.path.basename(curr_img_path),
            'cnn_distance': distance
        })

        if distance > threshold:
            subfolder_idx += 1
            subfolder_path = os.path.join(output_folder, f"sub{subfolder_idx}")
            os.makedirs(subfolder_path, exist_ok=True)

        # Save current image to current subfolder
        cv2.imwrite(os.path.join(subfolder_path, os.path.basename(curr_img_path)), cv2.imread(curr_img_path))
        prev_img_path = curr_img_path

    # Save log
    csv_path = os.path.join(output_folder, "cnn_diff_log.csv")
    pd.DataFrame(diff_log).to_csv(csv_path, index=False)
    print(f"✅ CNN difference log saved to: {csv_path}")
    print(f"✅ Grouped frames saved in: {output_folder}")


In [ ]:
import cv2
import numpy as np
import os
import csv

def run_optical_flow_on_folder(image_folder, output_csv_path):
    # Sort files numerically by extracting numbers from filenames
    def extract_frame_number(filename):
        import re
        match = re.search(r'(\d+)', filename)
        return int(match.group(1)) if match else -1

    image_files = sorted([
        os.path.join(image_folder, f)
        for f in os.listdir(image_folder)
        if f.lower().endswith(('.png', '.jpg', '.bmp', '.jpeg'))
    ], key=lambda x: extract_frame_number(os.path.basename(x)))

    if len(image_files) < 2:
        print(f"[{image_folder}] Not enough images to compute optical flow.")
        return

    # Initialize analysis results
    vertical_motion_magnitudes = []
    vertical_motion_rewarded_magnitudes = []
    vertical_motion_std_devs = []
    regionwise_std_devs = []

    frame1 = cv2.imread(image_files[0])
    prvs = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
    step = 16

    for i in range(1, len(image_files)):
        frame2 = cv2.imread(image_files[i])
        next = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)

        flow = cv2.calcOpticalFlowFarneback(prvs, next, None, 0.5, 3, 15, 3, 5, 1.2, 0)
        vert_flow = flow[..., 1]

        abs_vert_flow = np.abs(vert_flow)
        dynamic_threshold = 0.005 * np.max(abs_vert_flow)  

        # Mask for significant vertical motion
        moving_mask = abs_vert_flow > dynamic_threshold
        significant_motion_values = abs_vert_flow[moving_mask]

        total_pixels = abs_vert_flow.size
        num_moving_pixels = moving_mask.sum()
        coverage_ratio = num_moving_pixels / total_pixels

        # Parameters
        alpha = 2  # you can tune this

        if significant_motion_values.size > 0:
            weighted_motion = np.sum(significant_motion_values) / total_pixels
            rewarded_motion = weighted_motion * (coverage_ratio ** alpha)
            weighted_std = np.std(significant_motion_values)
        else:
            weighted_motion = 0
            rewarded_motion = 0
            weighted_std = 0

        # Save all
        vertical_motion_rewarded_magnitudes.append(rewarded_motion)
        vertical_motion_magnitudes.append(weighted_motion)  # or save both if you want
        vertical_motion_std_devs.append(weighted_std)

        # Grid-based regional std dev
        h, w = next.shape
        grid_size = 4
        block_h = h // grid_size
        block_w = w // grid_size

        region_means = []
        for gy in range(grid_size):
            for gx in range(grid_size):
                block = vert_flow[gy*block_h:(gy+1)*block_h, gx*block_w:(gx+1)*block_w]
                block_mean = np.mean(np.abs(block))
                region_means.append(block_mean)

        region_std = np.std(region_means)
        regionwise_std_devs.append(region_std)

        prvs = next.copy()

    # Save per-folder results
    with open(output_csv_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Frame Index", "Rewarded Vertical Motion", "Mean Vertical Motion", "Overall Std Dev", "Region-wise Std Dev"])
        for idx in range(len(vertical_motion_magnitudes)):
            writer.writerow([
                f"{idx} to {idx+1}",
                round(vertical_motion_rewarded_magnitudes[idx], 4),
                round(vertical_motion_magnitudes[idx], 4),
                round(vertical_motion_std_devs[idx], 4),
                round(regionwise_std_devs[idx], 4)
            ])
    print(f"✅ Saved: {output_csv_path}")

In [ ]:
## ------ Main Execution ------ ##

In [ ]:
# extract the frames from video into directory

video_path = "videos/lecture_demo2.mp4"
output_dir = "out_directory_lecture_demo2"
scenes = extract_and_save_scenes(video_path, output_dir)
print(f"Extracted and saved {len(scenes)} scenes.")


In [ ]:
# Group the frames by subdirectory based on the cnn values

input_dir = 'out_directory_lecture_demo2'       # input video frames folder
output_dir = 'lecture_demo2_subdir_cnn'         # where to save subfolders
threshold = 600                                 # try 600-1000 as starting point

group_frames_by_cnn_diff(input_dir, output_dir, threshold)


In [ ]:
# Compute the optical flow on each subdirectory

root_folder = 'lecture_demo2_subdir_cnn'  # change this to your output folder path
output_base = os.path.join(root_folder, 'motion_csvs')
os.makedirs(output_base, exist_ok=True)

for subfolder in sorted(os.listdir(root_folder)):
    sub_path = os.path.join(root_folder, subfolder)
    if os.path.isdir(sub_path) and subfolder.startswith("sub"):
        csv_path = os.path.join(output_base, f'vertical_motion_{subfolder}.csv')
        run_optical_flow_on_folder(sub_path, csv_path)

print("\n✅✅ All subfolders processed.")